# Cash-Karp Runge Kutta Method

### Import numpy and matplotlib

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

### Define the system of equations

In [2]:
def dydx(x,y):
    
    #create array
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    
    #set dzdx = -y
    y_derivs = -1*y[0]
    
    #return the array
    return y_derivs

### Define Cash-Karp Core

In [3]:
def cash_karp_core_mv(x_i,y_i,nv,h,f):
    
    #creating coefficients for adaptive step sizing
    ni = 7
    nj = 6
    ci = np.zeros(ni)
    aij = np.zeros( (ni,nj) )
    bi = np.zeros(ni)
    bis = np.zeros(ni)
    
    #values for ci, aij, bi, bis
    ci[2] = 1./5.
    ci[3] = 3./10.
    ci[4] = 3./5.
    ci[5] = 1.
    ci[6] = 7./8.
    
    aij[2,1] = 1./5.
    aij[3,1] = 3./40.
    aij[4,1] = 3./10.
    aij[5,1] = -11./54.
    aij[6,1] =  1631./55296.
    aij[3,2] = 9./40.
    aij[4,2] = -9./10.
    aij[5,2] = 5./2.
    aij[6,2] = 175./512.
    aij[4,3] = 6./5.
    aij[5,3] = -70./27.
    aij[6,3] = 575./13824.
    aij[5,4] = 35./27.
    aij[6,4] = 44275./110592.
    aij[6,5] = 253./4096.
    
    bi[1] = 37./378.
    bi[2] = 0.
    bi[3] = 250./621.
    bi[4] = 125./594.
    bi[5] = 0.
    bi[6] = 512./1771.
    
    bis[1] = 2825./27648.
    bis[2] = 0.
    bis[3] = 18575./48384.
    bis[4] = 13525./55296.
    bis[5] = 277./14336.
    bis[6] = 1./4. 
    
    #create k array
    ki = np.zeros((ni,nv))
    
    #find ki
    for i in range(1,ni):
        xn = x_i + ci[i]*h
        
        yn = y_i.copy()
        for j in range(1,i):
            yn += aij[i,j]*ki[j,:]
            
        ki[i,:] = h*f(xn,yn)
    
    ynpo = y_i.copy()
    ynpos = y_i.copy()
    
    for i in range(1,ni):
        ynpo += bi[i]*ki[i,:]
        ynpos += bis[i]*ki[i,:]
    
    Delta = np.fabs(ynpo-ynpos)
    
    return ynpo, Delta

### Define an adaptive step size driver for Cash-Karp

In [4]:
def cash_karp_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define a safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set max number of iterations
    imax = 10000
    
    #create an iteration variable 
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    #make the step
    h_step = h
    
    #adjust the step
    while(Delta.max()/tol > 1.0):
        
        yipo, Delta = cash_karp_core_mv(dydx,x_i,y_i,nv,h_step)
        
        #if error is too large
        if(Delta.max()/tol > 1.0):
            
            #we need to decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check the iteration
        if(i>=imax):
            print("Too many iteration in cash_karp_mv_ad()")
            raise StopIteration("Ending after i = ",i)
            
        #iterate 
        i += 1
        
    #try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, the nmew step and step we actually took
    return y_ipo, h_new, h_step

### Define a wraper for Cash-Karp

In [7]:
def cash_karp_mv(dfdx,a,b,y_a,tol):
    
    #dfdx id the derivate wrt x
    #a is the lower bound
    #b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #define starting step
    xi = a
    yi = y_a.copy()
    
    #create an initial step size
    h = 1.0e-4 * (b-a)
    
    #set the max number of iterations
    imax = 10000
    
    #set an interation variable
    i = 0
    
    #set # of ODEs to size of y_a
    nv = len(y_a)
    
    #set initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set a flag 
    flag = 1
    
    #loop until right side is reacher
    while(flag):
        
        #calculate y_i + 1
        yi_ipo, h_new, h_step = cash_karp_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update step
        h = h_new
        
        #prevent overshoot
        if(xi+h_step>b):
            
            h = b - xi
            #recalculate y_i+1
            yi_ipo, h_new, h_step = cash_karp_mv_ad(dydx,xi,yi,nv,h,tol)
            
            #break
            flag = 0
            
        #update values
        xi += h_step
        yi = yi_ipo.copy()
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_ipo= np.zeros((len(x),nv))
        y_ipo[0:len(x)-1,:] = y
        y_ipo[-1,:] = yi[:]
        del y
        y = y_ipo
        
        #prevent too many iterations
        if(i>=imax):
            
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number =",i)
            
        #iterate 
        i += 1
        
        #output information
        s = "i = %3d\tx = %9.8f\t = %9.8f\tb=%9.8f" % (i,xi,h_step,b)
        print(s)
        
        #break if new xi is == b
        if(xi==b):
            flag = 0
            
    #return results
    return x,y                

### Perform Integration

In [8]:
a = 0.0 
b = 2.0*np.pi 

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance =1.0e-6

#perform the integration
x,y = cash_karp_mv(dydx,a,b,y_0,tolerance)

TypeError: only integer scalar arrays can be converted to a scalar index